# Redes Convolucionales
Las redes convolucionales, o CNN (Convolutional Neural Network), son un tipo especializado de redes neuronales que han sido aplicado con mucho éxito en problemas en cuales los datos tienen forma de grillas, como son las imagenes. Se las conocen como redes convolucionales ya que aplican una operación matemática conocida como convolución. La convolución no es más que un operador móvil que se aplica repetidamente sobre los datos de entrada. Este operador está definido por una matríz pequeña, generalmente llamada kernel, que se aplica repetidamente sobre la imagen. Por ejemplo, imaginemos un kernel de 2 X 2.

$$K=\left[\begin{array}{cc}
k_{1,1} & k_{1,2}\\
k_{2,1} & k_{2,2}
\end{array}\right]$$

y una imagen de n X m:

$$I=\left[\begin{array}{cc}
i_{1,1} & i_{1,2} & ... & i_{1, m}\\
i_{2,1} & i_{2,2} & ... & i_{2, m}\\
... & ... & ... & ... \\
i_{n,1} & i_{n,2} & ... & i_{n, m}\\
\end{array}\right]$$

el resultado de aplicar la convolución sería:

$$C=\left[\begin{array}{cc}
c_{1,1} & c_{1,2} & ... & c_{1, m-1}\\
c_{2,1} & c_{2,2} & ... & c_{2, m-1}\\
... & ... & ... & ... \\
c_{n-1,1} & c_{n-1,2} & ... & c_{n-1, m-1}\\
\end{array}\right]$$

donde:

$$c_{i, j} = i_{i, j} * k_{1,1} + i_{i, j+1} * k_{1,2} + i_{i+1, j} * k_{2,1} + i_{i+1, j+2} * k_{2,2}$$

La operación de convolución ha sido usada con mucho exito en procesamiento de imagenes para detección de bordes, mejoramiento de imagenes, aplicación de blur, etc. Por ejemplo, Kirsch[1] propuso en 1971 una técnica que permite detectar estrucuras en las imagenes. Para esto, utiliza distintas [matrices de convolución](https://en.wikipedia.org/wiki/Kirsch_operator). Para ilustrar el la convolución utilizaremos $g^{(1)}$.

$$g^{(1)}=\left[\begin{array}{cc}
5 & 5 & 5 \\
-3 & 0 & -3 \\
-3 & -3 & -3
\end{array}\right]$$

[1] Kirsch, R. (1971). "[Computer determination of the constituent structure of biological images](https://www.sciencedirect.com/science/article/pii/0010480971900346)". Computers and Biomedical Research. 4: 315–328. doi:10.1016/0010-4809(71)90034-6.

In [ ]:
#Cargamos bibliotecas y demás
!pip install tqdm
#!pip install pil
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Concatenate, MaxPooling2D, BatchNormalization, GaussianNoise, GaussianDropout, Activation, Add
from tensorflow.keras.models import Model 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from cv2 import imread
from tqdm.notebook import tqdm
import os.path
import urllib.request
import os
import cv2
import time
import sys
from google.colab.patches import cv2_imshow


url = 'https://upload.wikimedia.org/wikipedia/commons/7/7b/R%C3%A9plica_de_la_piedra_movediza_de_Tandil.jpg'
filename = 'movediza.jpg'

def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                    (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()

# como vamos a necesitar descargar más archivos posteriormente, definimos una función que lo haga

def descargar_archivo(url,datapath):
    if not os.path.exists(datapath): # antes de descargar el archivo controlamos que no exista
       print("File does not exist")
  
       print("Downloading file...") 
       urllib.request.urlretrieve(url, datapath, reporthook)


descargar_archivo(url, filename)

In [ ]:
import cv2
#Cargo una imagen.
movediza = imread('movediza.jpg')
movediza = cv2.cvtColor(movediza, cv2.COLOR_BGR2RGB)
print(movediza.shape)
plt.imshow(movediza)
plt.show()

In [ ]:
#Cargo la matriz
kirsch_matrix = [[5, 5, 5], 
                [-3, 0, -3],
                [-3, -3, -3]]
kirsch_matrix = np.asarray(kirsch_matrix, dtype=np.float32) 
#Inicializo el placeholder como una matriz con garbage
movediza_k1 = np.empty((movediza.shape[0]-2, movediza.shape[1]-2, 3))
#Convolucion por fila, por columna, 
for i in tqdm(range(0, movediza.shape[0]-2)):
    for j in range(0, movediza.shape[1]-2):
        #Trato los canales de forma independiente
        for c in range(3): 
            # El operador * multiplica la matriz elemento a elemento y luego la
            # reduzco con una suma
            movediza_k1[i, j, c] = np.sum(movediza[i:i+3, j:j+3, c] * kirsch_matrix)
# Normalizo la imagen para poder mostrarla
movediza_k1 = (movediza_k1 - np.min(movediza_k1))/(np.max(movediza_k1) - np.min(movediza_k1))
plt.imshow(movediza_k1)
plt.show()

## ¡Ahora con keras!

In [ ]:
# De entrada tengo una matriz de 3 dimensiones, las primeras dos
# son de tamaño variable
i = Input(shape=(None, None, 3)) 
# Agrego una capa convolucional de 2 dimensiones
d = Conv2D(3, (3,3), activation='linear', use_bias=False)(i)
# Creo el modelo
kirsch = Model(inputs=i, outputs=d)
kirsch.summary()
# Compilo el modelo. No importa el loss ni optimizer.
kirsch.compile(loss='categorical_crossentropy', optimizer='sgd')
print('Forma de los parámetros de la convolución: {}'.format(kirsch.layers[1].kernel.shape))
# Cargo la matrix en un kernel apropiado
kirsch_kernel = np.zeros((3, 3, 3, 3))
for i in range(3):
    kirsch_kernel[:, :, i, i] = kirsch_matrix
# Utilizo el backend para cargar el kernel construido.
K.set_value(kirsch.layers[1].kernel, kirsch_kernel)
# Aplico la convolución.
movediza_k2 = kirsch.predict(np.asarray([movediza]))[0]
# Normalizo la imagen para poder mostrarla
movediza_k2 = (movediza_k2 - np.min(movediza_k2))/(np.max(movediza_k2) - np.min(movediza_k2))
plt.imshow(movediza_k2)
plt.show()

## Explicación del código
Como primera etapa, creo la estructura de la CNN. En particular es de interes la creación de la capa convolucional 2D:

```d = Conv2D(3, (3,3), activation='linear', use_bias=False)(i)```

El primer parámetro indica cuantos canales tiene la imagen de salida. En este caso, como el objetivo es una imagen RGB la cantidad de canales es 3. El segundo parámetro indica que función de activación se aplica sobre la convolución. En este caso, se aplica la función lineal/identidad $f(x)=x$. Finalmente, indicamos que no se utilizará un parametro de bías, equivalente al $b$ en la regresión lineal o logística. 

Como segunda etapa, compilamos el modelo. Se especifican función de perdida y optimizador porque son obligatorios, pero solo son interesa compilar por lo que estos parámetros pueden ser cualquiera.

```kirsch = Model(inputs=i, outputs=d)
kirsch.summary()
kirsch.compile(loss='categorical_crossentropy', optimizer='sgd')```

A diferencia de la matriz original de 3x3, la matriz kernel de la capa convolucional es de 3x3x3x3. Esto se debe a que la operación de convolución en este caso es más generico. Las primeras dos dimensiones efectivamente se corresponde con las dimensiones de la convolución. La tercera dimensión se corresponde con la cantidad de canales de la imagen, ya que la convolución en este caso tambíen considera los canales. Finalmente, la cuarta dimensión se corresponde con la cantidad de canales de salida.

```kirsch_kernel = np.zeros((3, 3, 3, 3))
for i in range(3):
    kirsch_kernel[:, :, i, i] = kirsch_matrix
K.set_value(kirsch.layers[1].kernel, kirsch_kernel)```

Como queremos que la convolución se aplice por cada canal, lo único que hay que hace es asignarle la matriz de convolución al canal correspondiente, dejando todos los otros valores en cero. Como resultado, se ignoran los valores en los otros canales, ya que se multiplican por cero (elemento absorbente para la multiplicación) y luego se suman (elemento neutro en la suma).

## Intuiciones
Como se mostró en el ejemplo anterior, los filtros/kernels convolucionales pueden aplicarse para detectar ciertas características de las imagenes como son los bordes. Además, estos kernels tiene relativamente pocos parámetros. En este caso, la cantidad de parámetros eran 81 (3x3x3x3). Si se aplicara una red densa sobre la imagen de entrada de 640x480x3 y se espera una salida de aproximadamente el mismo tamaño se requeririan 8.4924656e11 ((640x480x3)^2) parámetros.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

print('Cargando el dataset')
(x_train, y_train), (x_test, y_test) = mnist.load_data()
size = x_train.shape[1]*x_train.shape[2]
x_train = np.expand_dims(x_train, axis=-1) / 255
x_test = np.expand_dims(x_test, axis=-1) / 255

print(x_train.shape)
print(x_train.shape[1:])

In [ ]:
i = Input((x_train.shape[1:]))
d = Conv2D(3, (3,3), activation='relu')(i)
d = Conv2D(5, (5,5), activation='relu')(d)
d = Conv2D(5, (3,3), activation='relu')(d)
d = Conv2D(5, (3,3), activation='relu')(d)
d = Flatten()(d)
d = Dense(10, activation='softmax')(d)
model = Model(i, d)
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.fit(x_train, to_categorical(y_train), epochs=10, 
          validation_data=(x_test, to_categorical(y_test)))


## ¿Qué ven las capas?

La idea qe que las capas más cercanas al input capturan caracteristicas simples, como bordes y gradientes de color, mientras que más abajo en la red capturan caracteristicas más complejas.

In [ ]:
idx = 6
print('Número de ejemplo')
ax = plt.subplot(111)
ax.matshow(x_test[idx, :, :, 0], cmap='gray')
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.show()
#Ponga su código para ver las capas intermedias de su modelo aquí

In [ ]:
max_depth = 3

pred = model.layers[1](x_test[idx:idx+1, ...])
for i in range(2, max_depth+1):
    pred = model.layers[i](pred)
pred = pred[0]

In [ ]:
print(pred.shape)
plt.matshow(x_test[idx, :,:,0])
plt.matshow(pred.numpy()[:,:,0])
plt.matshow(pred.numpy()[:,:,1])
plt.matshow(pred.numpy()[:,:,2])
plt.show()

### Pooling
Pooling es una operación que reduce la salida de una red en una posición utilizando alguna función estádistica de la misma. Por ejemplo, *Max pooling* reemplaza una posición y sus vecinos por el máximo valor en el vecindario. Keras proporciona diversas capas de pooling.
* `MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)`: Obtine el maximo de un vecindario de tamaño `pool_size`, moviendose en la imagen utilizando los desplazamientos indicados en `strides` (si es `Nono` `strides=pool_size`).
* `AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)`: Obtine el maximo de un vecindario de tamaño `pool_size`, moviendose en la imagen utilizando los desplazamientos indicados en `strides` (si es `Nono` `strides=pool_size`).
* `GlobalMaxPooling2D(data_format=None)`: Calcula el máximo por canal, transformando una entrada de forma `(batch_size, rows, cols, channels)` a una salida de forma `(batch_size, channels)`.
* `GlobalAveragePooling2D(data_format=None)`: Calcula el promedio por canal, transformando una entrada de forma `(batch_size, rows, cols, channels)` a una salida de forma `(batch_size, channels)`.

## Overfitting
Overfitting es un fenómeno que ocurre cuando el modelo entrenado se ajusta tan bien a los datos de entrenamiento, que falla al generalizar para nuevos datos. Este es un problema muy común en las redes neuronales ya que estas tienen mucha capacidad para adaptarse a los datos de entrenamiento. En general, cuantos más parámetros entrenables tenga una red neuronal es más probable que sufra algún tipo de overfitting.


In [ ]:
from tensorflow.keras.datasets import cifar100 
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

#y_train = np.reshape(y_train, (y_train.shape[0]))
#y_test = np.reshape(y_test, (y_test.shape[0]))


print('100 primeros elementos del conjunto de entrenaimento')
f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(x_train[i + j*10, :, :], cmap='gray')
plt.show()


x_train = x_train / 255
x_test = x_test / 255

In [ ]:
i = Input((x_train.shape[1:]))
d = Conv2D(3, (3,3), activation='relu')(i)
d = Conv2D(5, (5,5), activation='relu')(d)
d = Conv2D(5, (3,3), activation='relu')(d)
d = Conv2D(5, (3,3), activation='relu')(d)
d = Flatten()(d)
d = Dense(100, activation='softmax')(d)
model = Model(i, d)
model.summary()
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy', 'sparse_top_k_categorical_accuracy'])

model.fit(x_train, y_train, epochs=10, 
          validation_data=(x_test, y_test))

In [ ]:
i = Input((x_train.shape[1:]))
d = Conv2D(32, (3,3), activation='relu')(i)
d = Conv2D(32, (5,5), activation='relu')(d)
d = Conv2D(64, (3,3), activation='relu')(d)
d = Conv2D(128, (3,3), activation='relu')(d)
d = Flatten()(d)
d = Dense(100, activation='softmax')(d)
model = Model(i, d)
model.summary()
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy', 'sparse_top_k_categorical_accuracy'])

model.fit(x_train, y_train, epochs=10, 
          validation_data=(x_test, y_test))

In [ ]:
model.layers[3].kernel.shape

In [ ]:
def inception(x, filters=16):
    x_1_1 = Conv2D(filters, (1, 1), padding='same')(x)

    x_3_3 = Conv2D(filters, (3, 3), padding='same')(x)
    x_3_3 = Conv2D(filters, (1, 1), padding='same')(x_3_3)
    
    x_5_5 = Conv2D(filters, (5, 5), padding='same')(x)
    x_5_5 = Conv2D(filters, (1, 1), padding='same')(x_5_5)

    m_3_3 = Conv2D(filters, (1, 1), padding='same')(x)
    m_3_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(m_3_3)

    x = Concatenate()([x_1_1, x_3_3, x_5_5, m_3_3])
    #x = BatchNormalization(epsilon=1e-5)(x)
    return Activation('relu')(x)


i = Input((x_train.shape[1:]))
d = GaussianDropout(0.2)(i)
d = inception(d)
d = inception(d)
d = MaxPooling2D()(d)
d = GaussianDropout(0.2)(d)
d = inception(d, 32)
d = Flatten()(d)
d = Dense(100, activation='softmax')(d)
model = Model(i, d)
model.summary()
model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy', 'sparse_top_k_categorical_accuracy'])

model.fit(x_train, y_train, epochs=10, 
          validation_data=(x_test, y_test))

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
def block1(x, filters, kernel_size=3, stride=1,
           conv_shortcut=True, name=None):

    bn_axis = 3 

    if conv_shortcut is True:
        shortcut = Conv2D(4 * filters, 1, strides=stride,
                                 name=name + '_0_conv')(x)
        shortcut = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                             name=name + '_0_bn')(shortcut)
    else:
        shortcut = x

    x = Conv2D(filters, 1, strides=stride, name=name + '_1_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_1_bn')(x)
    x = Activation('relu', name=name + '_1_relu')(x)

    x = Conv2D(filters, kernel_size, padding='SAME',
                      name=name + '_2_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_2_bn')(x)
    x = Activation('relu', name=name + '_2_relu')(x)

    x = Conv2D(4 * filters, 1, name=name + '_3_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_3_bn')(x)

    x = Add(name=name + '_add')([shortcut, x])
    x = Activation('relu', name=name + '_out')(x)
    return x


i = Input((x_train.shape[1:]))
d = block1(i, 32, name='block1')
d = MaxPooling2D()(d)
d = block1(d, 64, name='block2')
d = MaxPooling2D()(d)
d = block1(d, 128, name='block3')
d = Flatten()(d)
d = Dense(100, activation='softmax')(d)
model = Model(i, d)
model.summary()
model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy', 'sparse_top_k_categorical_accuracy'])

model.fit(x_train, y_train, epochs=10, 
          validation_data=(x_test, y_test))

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

# Utilización de modelos preentrenados

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input

model = VGG16()

model.summary()

In [ ]:
model2 = VGG16(include_top=False)
model2.summary()

In [ ]:
i = Input((None, None, 3))
d = model2(i)
fc1 = Conv2D(4096, (7,7), activation='relu')(d)
fc2 = Conv2D(4096, (1,1), activation='relu')(fc1)
p = Conv2D(1000, (1,1), activation='softmax')(fc2)

m = Model(i, p)
m.compile(loss='categorical_crossentropy', optimizer='nadam')

K.set_value(m.layers[-3].kernel, np.reshape(K.get_value(model.layers[-3].kernel), m.layers[-3].kernel.shape))
K.set_value(m.layers[-3].bias, np.reshape(K.get_value(model.layers[-3].bias), m.layers[-3].bias.shape))

K.set_value(m.layers[-2].kernel, np.reshape(K.get_value(model.layers[-2].kernel), m.layers[-2].kernel.shape))
K.set_value(m.layers[-2].bias, np.reshape(K.get_value(model.layers[-2].bias), m.layers[-2].bias.shape))

K.set_value(m.layers[-1].kernel, np.reshape(K.get_value(model.layers[-1].kernel), m.layers[-1].kernel.shape))
K.set_value(m.layers[-1].bias, np.reshape(K.get_value(model.layers[-1].bias), m.layers[-1].bias.shape))

model = m
del m

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
minivan = 656 #705

#url = 'https://www.autocar.co.uk/sites/autocar.co.uk/files/styles/gallery_slide/public/images/car-reviews/first-drives/legacy/large-2479-s-classsaloon.jpg?itok=QTxMln2k'
#url = 'https://images.squarespace-cdn.com/content/v1/59cf0541d55b41b151ace075/1575580994414-FLMU5ZJEOB3I1OLY2VOZ/ke17ZwdGBToddI8pDm48kHoY9blY_oPV5Bzcqi3t-roUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKcx5yuVS_0jJ0plzpb6sRSkaYELcEKb0geq5qayGSPLYpUQGlT0O5pGW3qo4jTaCvA/Cincinnati_Street_View_Image.jpg?format=2500w'
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/7/7d/Anawrahta_road_traffic.jpg/1599px-Anawrahta_road_traffic.jpg'
url = 'https://urbanfarmsuganda.files.wordpress.com/2020/03/traffic-jam.jpg'
filename = 'cars.jpg'

descargar_archivo(url, filename)

cars = imread('cars.jpg')
cars = cv2.cvtColor(cars, cv2.COLOR_BGR2RGB)
print(cars.shape)
plt.imshow(cars)
plt.show()

pred = model.predict(np.expand_dims(preprocess_input(cars), 0))[0, :, :, :]
print(pred.shape)

plt.imshow(pred[:, :, minivan])
plt.show()
plt.imshow((cars * np.expand_dims(cv2.resize(pred[:, :, minivan], cars.shape[0:2][::-1]), axis=-1)/np.max(pred[:, :, minivan])).astype(np.uint8))
plt.show()


In [ ]:
classes = [407, 436, 468, 511, 565, 609, 627, 656, 705, 751, 817]
p_max = np.max(pred[:, :, classes], axis=-1)
plt.imshow(p_max)
plt.show()
plt.imshow((cars * np.expand_dims(cv2.resize(p_max, cars.shape[0:2][::-1]), axis=-1)).astype(np.uint8))
plt.show()

In [ ]:
from skimage.transform import pyramid_gaussian
from skimage import color
from imutils.object_detection import non_max_suppression
from collections import deque
import imutils
import numpy as np
import cv2
from tensorflow.keras.applications import VGG19


descargar_archivo('https://users.exa.unicen.edu.ar/~jmrodri/car3b2.jpg', 'car3b2.jpg')
model = VGG19()

# Generador de ventana deslizante 
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])


img= cv2.imread("car3b2.jpg")

preproc_img = preprocess_input(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

#Acá agrego los boxes detectados
detections = []

 
#Tamaño de las imagenes en el dataset de training
windowSize=(224, 89)
downscale=1.5

for scale, resized in enumerate(pyramid_gaussian(preproc_img, multichannel=True, downscale=downscale)): 
    print('Scale: {}'.format(scale))
    imgs = []
    x_y = []
    for (x,y,window) in sliding_window(resized, stepSize=10, windowSize=windowSize):
        if window.shape[0] != windowSize[1] or window.shape[1] !=windowSize[0]:
            continue
        imgs.append(cv2.resize(window, (224, 224)))
        x_y.append((x, y))
    print('Predicting')
    if len(imgs) == 0:
        break
    print(len(imgs))
    imgs = np.asarray(imgs)
    predictions = decode_predictions(model.predict(imgs, verbose=1, batch_size=100))
    for i, (decoded, (x, y)) in enumerate(zip(predictions, x_y)):
        if decoded[0][1] == 'minivan' and decoded[0][2] > .8:
                '''cv2_imshow(imgs[i, ...])
                print("Detection:: Location -> ({}, {})".format(x, y))
                print("Scale ->  {} | Confidence Score {} \n".format(scale,pred))'''
                detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), [decoded[0][2]],
                                   int(windowSize[0]*(downscale**scale)), 
                                      int(windowSize[1]*(downscale**scale))))  

    

for (x_tl, y_tl, scale, w, h) in detections:
    cv2.rectangle(img, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 255), thickness = 2)    
    size = cv2.getTextSize('Prob: {:0.2f}%'.format(scale[0]*100),
                           fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1.5, 
                           thickness=1)[0]
    cv2.rectangle(img, (x_tl + 5, y_tl + h - 5 - size[1]), 
                  (x_tl + 5 + size[0], y_tl + h - 2), (0, 0, 0), thickness = -1)
    cv2.putText(img, text='Prob: {:0.2f}%'.format(scale[0]*100),
                org=(x_tl+3, y_tl + h - 3),
                fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1.5, color=(255,255,255))
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections])
sc = [score[0] for (x, y, score, w, h) in detections]
print("detection confidence score: ", sc)
sc = np.array(sc)
pick = non_max_suppression(rects, probs = sc, overlapThresh = 0.1)

#Bounding boxes seleccionados        
for (xA, yA, xB, yB) in pick:
    cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)
cv2_imshow(img)

## Modelos para esto

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_hub as hub


module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

In [ ]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...], img


result = detector(load_img('car3b2.jpg')[0])
print(result)

In [ ]:
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import random

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())
  random.shuffle(colors)

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

In [ ]:
def run_detector(detector, path):
    img, imgOrg = load_img(path)
    start_time = time.time()
    result = detector(img)
    end_time = time.time()

    result = {key:value.numpy() for key,value in result.items()} #Extraigo los valores de TF a numpy

    print("%d Objetos encontrados." % len(result["detection_scores"]))
    print("Tiempo de la inferencia: ", end_time-start_time)

    image_with_boxes = draw_boxes(
        imgOrg.numpy(), result["detection_boxes"],
        result["detection_class_entities"], result["detection_scores"])

    display_image(image_with_boxes)


run_detector(detector, 'car3b2.jpg')

# Face recognition

In [ ]:
!pip install face_recognition

In [ ]:
!wget https://users.exa.unicen.edu.ar/~jmrodri/faces.zip
!unzip faces.zip

In [ ]:
import face_recognition as fr
import cv2
import numpy as np


def encodings(names):
    enc = []
    for n in names:
        img = fr.load_image_file('{}.jpg'.format(n.lower()))
        face = fr.face_encodings(img, fr.face_locations(img))
        enc.append(face[0])
    return enc

names = ['Rachel', 'Ross', 'Monica', 'Phoebe','Joey']

known = encodings(names)

img = fr.load_image_file('ad.jpg')
face_loc = fr.face_locations(img)
face_enc = fr.face_encodings(img, face_loc)
res = ''

ad_names = []
for enc in face_enc:
    name = 'Unknown'
    match = fr.compare_faces(known, enc, tolerance=0.6)
    dist = fr.face_distance(known, enc)
    idx = np.argmin(dist)
    if match[idx]:
        name = names[idx]
    ad_names.append(name)

img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
for (top, right, bottom, left), name in zip(face_loc, ad_names):
        cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)

        cv2.rectangle(img, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(img, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

cv2_imshow(img)

In [ ]:
known